In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR

from torch.utils.data import DataLoader
from BookDataset import getBookDataset, BookDataset
from embeddings import EmbeddingSummation

In [2]:
train, val, test, featureSizes = getBookDataset('../Data_Train.xlsx', '../Data_Test.xlsx')

In [3]:
class BookPrice(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config
        outDims = lambda x: int(x**.25) + 1
        self.outDims = outDims
        
        # embeddings for categorical variables: Author, Genre, BookCategory
        self.AuthorEmbedding = nn.Embedding(config.Author,  outDims(config.Author))
        self.GenreEmbedding  = nn.Embedding(config.Genre,  outDims(config.Genre))
        self.BookCategoryEmbedding  = nn.Embedding(config.BookCategory,  outDims(config.BookCategory))
        categoricalUnits = 5 * outDims(config.Author) + outDims(config.Genre) + outDims(config.BookCategory)
        # ------
        
        
        # embeddings for text features Title, Synopsis
        self.TitleEmbedding    = EmbeddingSummation()
        self.SynopsisEmbedding = EmbeddingSummation()
        textUnits = 128 * 2
        #-------
        
        # Ratings, Reviews and Edition
        numericUnits = 3
        #-------

        total = categoricalUnits + textUnits + numericUnits
        
        self.Dense = nn.Sequential(
            nn.Linear(total, 64), nn.LayerNorm(64), nn.Tanh(),
            nn.Linear(64, 32), nn.LayerNorm(32), nn.Tanh(), 
            nn.Linear(32, 1)
        )

    def forward(self, batch:dict) -> torch.Tensor:
        authors = torch.stack([ batch[f'author_{i}'] for i in range(5) ]).T
        authDim = 5 * self.outDims(self.config.Author)
        categoricals = [
            self.AuthorEmbedding(authors).reshape(-1, authDim),
            self.GenreEmbedding(batch['Genre']),
            self.BookCategoryEmbedding(batch['BookCategory'])
        ]
        
        text = [
            self.TitleEmbedding(batch['Title']),
            self.SynopsisEmbedding(batch['Synopsis'])
        ]
        
        numericals  = [torch.stack([
            batch['Ratings'],
            batch['Reviews'],
            batch['Edition']
        ]).T]
        
        inputs = torch.cat(categoricals + text + numericals, dim=-1)
        
        logits = self.Dense(inputs)
        return logits
    
    
    def training_step(self, batch:dict, batchIdx:int)->torch.Tensor:
        logits = self(batch)

        loss = F.mse_loss(logits, batch['Price'].reshape(-1, 1))
        self.log('loss', loss)

        return loss

    def validation_step(self, batch:dict, batchIdx:int):
        logits = self(batch)
        loss = F.mse_loss(logits, batch['Price'].reshape(-1, 1))
        self.log('val_loss', loss)
        return score

    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr=1e-3)
        scheduler = LambdaLR(optimizer, lr_lambda=lambda epoch: 0.99 * epoch)
        return [optimizer], [scheduler]

    
model = BookPrice(featureSizes)
# loader = DataLoader(BookDataset(train), batch_size=8)
# batch = next(iter(loader))
# with torch.no_grad():
#     print(model.training_step(batch, 0))
x = sum(p.numel() for p in model.parameters())
print(f'BookPrice model has {x:,} parameters')

BookPrice model has 7,836,458 parameters


In [4]:
stopping = EarlyStopping(monitor='val_loss', patience=10, mode='max')
ckpt = ModelCheckpoint(dirpath='chekpoints', filename='{epoch}-{val_loss:.5f}',
                       monitor='val_score', mode='max',
                      save_weights_only=True)

trainer = pl.Trainer(gpus = 1, callbacks=[stopping, ckpt])

loaders = []
for i, data in enumerate([train, val, test]):
    shuffle = i==0
    loaders.append( DataLoader(BookDataset(data), batch_size=32, shuffle=shuffle ) )

trainer.fit(model, loaders[0], loaders[1])

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                  | Type               | Params
-------------------------------------------------------------
0 | AuthorEmbedding       | Embedding          | 2.6 K 
1 | GenreEmbedding        | Embedding          | 804   
2 | BookCategoryEmbedding | Embedding          | 24    
3 | TitleEmbedding        | EmbeddingSummation | 3.9 M 
4 | SynopsisEmbedding     | EmbeddingSummation | 3.9 M 
5 | Dense                 | Sequential         | 20.9 K
-------------------------------------------------------------
7.8 M     Trainable params
0         Non-trainable params
7.8 M     Total params


1

In [5]:
trainer.test(model, loaders[2])

c:\users\deepak h r\appdata\local\programs\python\python37\lib\site-packages\pytorch_lightning\utilities\distributed.py:49: UserWarning: you passed in a test_dataloader but have no test_step. Skipping test loop
  warnings.warn(*args, **kwargs)


1

In [12]:
results = []
model = model.eval()
with torch.no_grad():
    for batch in loaders[2]:
        for key in batch:
            if key not in 'Title Synopsis'.split():
                batch[key] = batch[key].to(model.device)
        preds = model(batch)
        results.append(preds)
results = torch.cat(results).cpu().numpy().reshape(-1)

In [13]:
submission = pd.DataFrame({
    'Price': np.exp(results) - 1
})
print("Saving submission ", submission.shape)
submission.to_excel('mysubmission_tanh_last.xlsx', index=False)

Saving submission  (1560, 1)


In [20]:
model = BookPrice.load_from_checkpoint(ckpt.best_model_path, config=featureSizes).eval()
model

BookPrice(
  (AuthorEmbedding): Embedding(521, 5)
  (GenreEmbedding): Embedding(201, 4)
  (BookCategoryEmbedding): Embedding(12, 2)
  (TitleEmbedding): EmbeddingSummation(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
  )
  (SynopsisEmbedding): EmbeddingSummation(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
  )
  (Dense): Sequential(
    (0): Linear(in_features=290, out_features=64, bias=True)
    (1): LayerNorm((64,), eps=1e-05, elementwise_affi

In [22]:
results = []
with torch.no_grad():
    for batch in loaders[2]:
        for key in batch:
            if key not in 'Title Synopsis'.split():
                batch[key] = batch[key].to(model.device)
        preds = model(batch)
        results.append(preds)
results = torch.cat(results).cpu().numpy().reshape(-1)
submission = pd.DataFrame({
    'Price': np.exp(results) - 1
})
print("Saving submission ", submission.shape)
submission.to_excel('mysubmission_tanh.xlsx', index=False)

Saving submission  (1560, 1)
